Oblv Client is from our PyOblv package.
ds_abc is the PySyft Client for the Data Scientist on domain abc
ds_xyz is the PySyft Client for the Data Scientist on domain xyz

In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import oblv
canada = sy.login(url="localhost",port=8081,email="sheldon@caltech.edu",password="changethis")
italy = sy.login(url="localhost",port=8082,email="sheldon@caltech.edu",password="changethis")


Anyone can login into your account right now because your password is the default PySyft password!!!

Connecting to localhost... done! 	 Logging into canada... done!

Anyone can login into your account right now because your password is the default PySyft password!!!

Connecting to localhost... done! 	 Logging into italy... done!


In [ ]:
# For real enclave deployment
# depl = sy.create_deployment([canada,italy],"demo_openmined_1",key_name = "new6",infra="r5.xlarge")
# depl.deployment_id

In [2]:
# To use an already existing enclave , local mode deployment
depl = sy.oblv.model.DeploymentClient(deployment_id="d-2dfedbb1-7904-493b-8793-1a9554badae7",oblv_client=None,domain_clients=[canada,italy],user_key_name="first") #connection_port key can be added to set the port on which oblv_proxy will run
depl.deployment_id

'd-2dfedbb1-7904-493b-8793-1a9554badae7'

In [ ]:
# To check current state of enclave in real deployment
# depl_info = oblv_client.deployment_info(depl.deployment_id)
# print(depl_info.current_state)

In [3]:
canada_ptr = canada.store[-1]
italy_ptr = italy.store[-1]

In [4]:
depl.__dict__

{'deployment_id': 'd-2dfedbb1-7904-493b-8793-1a9554badae7',
 'user_key_name': 'first',
 'oblv_client': None,
 'client': [<DomainClient - canada: <UID: 93a8b1d1e5ce44f0b9602051cb29bb09>>,
  <DomainClient - italy: <UID: 1426192a91004baaae0c68600c1f0997>>],
 '_DeploymentClient__conn_string': '',
 '_DeploymentClient__process': None,
 '_DeploymentClient__logs': None}

tensor_abc and tensor_xyz are the two tensors selected from two different domains.

Creating a deploynent now, using both the domains (abc and xyz).

The public-private key pair was already generated for both these domains, but if it is not, the admins of the domain can use the method **create_key** in oblv api to create one.

Deployment with shown id is created. The status of the deployment can be checked with the next block. Once the status is **Running**, the further steps can be taken after an initialization of the connection. 

The port on which the connection is to be made can be specified either during the time of creation of deployment or can be later updated as shown below.

*Do not change the connection port after initializing the connection*

In [ ]:
canada.oblv.check_connection(depl)

In [5]:
depl.check_proxy_running()

False

In [6]:
depl.initiate_connection(3030)

In [7]:
canada_dataset = canada.oblv.transfer_dataset(deployment=depl,dataset=canada_ptr)
italy_dataset = italy.oblv.transfer_dataset(deployment=depl,dataset=italy_ptr)

In [8]:
depl.get_uploaded_datasets()

[{'id': 'b11bacf72c6345dcae4d3664764a95b8', 'owner': 'canada'},
 {'id': 'b011c8026286466b999d0d127f56728b', 'owner': 'italy'}]

In [9]:
type(canada_dataset)

syft.oblv.oblv_enclave_pointer.OblvEnclavePointer

In [10]:
dataset_result = canada_dataset + italy_dataset

In [11]:
dataset_sum_result = dataset_result.sum(keepdims=True)
# dataset_result = sy.OblvTensorWrapper(id="r-3533056d-e408-40bc-8550-d6d98d7cfca3",deployment_client=depl)

In [12]:
dataset_sum_result

In [13]:
p_id=dataset_sum_result.request_publish(sigma=1000)
print(p_id)

p-7063208c-d458-48e5-a228-a6e9a28526e1


In [14]:
print(canada.privacy_budget)
print(italy.privacy_budget)

9999999.0
9999999.0


In [15]:
depl.check_publish_request_status(p_id)

Result is ready


In [16]:
print(canada.privacy_budget)
print(italy.privacy_budget)

9999998.94627288
9999998.94627288


In [17]:
result = depl.fetch_result(p_id)
print(result)

[866.0016337604535]


In [ ]:
# To be used in real deployment, does not work in enclave emulation
depl.close_connection()
oblv_client.remove_deployment(depl.deployment_id)

In [ ]:
depl.fetch_current_proxy_logs()

In [19]:
italy.oblv.get_key()

'MIIBCgKCAQEAwDtMEYyQwL531HyGB3kLUrr5L3FJCpptaYqOUq4qHopqEZnvIg+pBHuarwqLF5Pu8ltO9tPKDyCu7KwjTj0wI5G4jFFqj9iofGgny6439Y6O9kcA3v8Ht+PJmlCLVFPee9wmFMBs+AFRH4hbR8ENO2O4KTgiuztsXnTxAOe10scpUb9NutTUy+XytPV1bQN1uQ0QLLIHgJH4YWXnHBvu9SqVT4abvb/nHDEgNuE8XV2xQRs5+ivpqfxKrcGVgfVpbinZId/OAToPbRIbTwx4Gj/McNpxxW4DB1TZywrucmqnFjfCzHatmy8KgJ49g3q/of4bXuyoT81YU4A5eo5MHQIDAQAB'

Known Issues - 

Large dataset cannot be used as of now, cause they go in long re-runs for cache calculation.

Proxy sends 200 status code while the enclave returns 400.

Mean method when performed on tensor returns a tensor with data_subjects not of type of array and it fails when we perform publish on it.
